# PhenoAge

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Define required clock parameters

Let's define some required information first:

In [2]:
clock_name = 'phenoage'
data_type = 'blood chemistry'
model_class = 'LinearModel'
species = 'Homo sapiens'
year = 2018
approved_by_author = '⌛'
citation = "Levine, Morgan E., et al. \"An epigenetic biomarker of aging for lifespan and healthspan.\" Aging (albany NY) 10.4 (2018): 573."
doi = "https://doi.org/10.18632%2Faging.101414"
notes = None

## Download necessary data

In [3]:
# No need; go to paper for coefficients

## Load data

In [4]:
features = [
    "albumin",
    "creatinine",
    "glucose",
    "log_crp",
    "lymphocyte_percent",
    "mean_cell_volume",
    "red_cell_distribution_width",
    "alkaline_phosphatase",
    "white_blood_cell_count",
    "age"
]

coefs = [
    -0.0336,
    0.0095,
    0.1953,
    0.0954,
    -0.0120,
    0.0268,
    0.3306,
    0.0019,
    0.0554,
    0.0804,
]

## Extract features and weights

First, let's extract the features and weights:

In [5]:
weights = torch.tensor(coefs).unsqueeze(0)
intercept = torch.tensor([-19.9067])

## Load weights into pyaging model

#### Linear model

In [6]:
model = pya.models.LinearModel(input_dim=len(features))

model.linear.weight.data = weights.float()
model.linear.bias.data = intercept.float()

model

LinearModel(
  (linear): Linear(in_features=10, out_features=1, bias=True)
)

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [7]:
reference_feature_values = None

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format, with all items required. It takes in x in the form of a numpy array.

In [8]:
preprocessing = None

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [9]:
def postprocessing_function(x):
    """
    Applies a convertion from a CDF of the mortality score from a Gompertz
    distribution to phenotypic age.
    """
    # lambda
    l = 0.0192
    mortality_score = 1 - torch.exp(-torch.exp(x) * (torch.exp(120 * l) - 1) / l)
    age = 141.50225 + torch.log(-0.00553 * torch.log(1 - mortality_score)) / 0.090165
    return age
    
postprocessing_function_string = marshal.dumps(postprocessing_function.__code__)

postprocessing_helper_objects = None

postprocessing = {
    'name': 'mortality_to_phenoage',
    'postprocessing_function': postprocessing_function_string,
    'postprocessing_helper_objects': postprocessing_helper_objects
}

## Check all data objects

Let's print all data objects to check if they make sense.

#### features

In [10]:
def my_print_function():
    print(f"There are {len(features)} features.")
    print(features)
pya.utils.print_to_scrollable_output(my_print_function)

#### reference_feature_values

In [11]:
def my_print_function():
    if reference_feature_values:
        print(f"There are {len(reference_feature_values)} reference feature values.")
    print(reference_feature_values)
pya.utils.print_to_scrollable_output(my_print_function)

#### preprocessing

In [12]:
def my_print_function():
    print(preprocessing)
    if preprocessing:
        print(preprocessing_helper_objects)
pya.utils.print_to_scrollable_output(my_print_function)

#### postprocessing

In [13]:
def my_print_function():
    print(postprocessing)
    if postprocessing:
        print(postprocessing)
pya.utils.print_to_scrollable_output(my_print_function)

#### weight_dict

In [14]:
def my_print_function():
    for name, param in model.named_parameters():
        print(f"Layer: {name}")
        print(f"Shape: {param.shape}")
        print(param.data)
pya.utils.print_to_scrollable_output(my_print_function)

## Write clock dictionary

Let's put everything together and save:

In [15]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory

Delete all files that are not clock jupyter notebooks:

In [16]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted folder: .ipynb_checkpoints
